<a href="https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/ModelEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project - Person classification task evaluation

[https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/ModelEvaluation.ipynb](https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/ModelEvaluation.ipynb)

Importing from Google Drive the dataset.zip and extract into dataset folder, change the path with your dataset location

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/dataset.zip" -d dataset

Importing necessary libraries

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
import csv
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter

# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

In [3]:
'''
Returns: dict where keys are the labels associated with their encoding and the values are the corresponding indexes in the one-hot encoding
Input arguments
  annotations_frame: dataframe containing annotations associated to people ids
'''
def setupLabelsDict(annotations_frame):
    labels = {}
    index = 0
    for i in list(annotations_frame):
        # id is not part of labels
        if(i != "id"):
            # to build the one-hot encoding I need to know the min and max value range for each label
            for j in range(min(annotations_frame[i]), max(annotations_frame[i])+1):
                labels[f"{i}-{j}"] = index
                index+=1
    return labels

In [4]:
class PeopleTestDataset(Dataset):
    """People test dataset containing only images."""

    def __init__(self, root_dir):
        """
        Args:
            root_dir: Directory with all the images.
        """
        self.root_dir = root_dir
        self.img_files = [f for f in listdir(root_dir)]
    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread("%s/%s" % (self.root_dir, self.img_files[idx]))
        image = T.ToTensor()(image)
        image = F.interpolate(image, size=128)  
        return (image, self.img_files[idx]) 

In [13]:
'''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

def initialize_resnet(num_classes):
  # load the pre-trained ResNet-50
  resnet = torchvision.models.resnet50(pretrained=True, progress=False)
  num_features = resnet.fc.in_features
  resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=num_features, out_features=1024),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.Linear(in_features=512, out_features=num_classes),
    torch.nn.Sigmoid()
  )
  return resnet

In [6]:
"""
Returns: 
  test_loader: DataLoader used during testing
Input arguments
  img_root: directory containing the dataset images
  test_batch_size: batch size used during validation and test phase
"""
def get_data(img_root, test_batch_size=1):
  # Load data
  test_data = PeopleTestDataset(root_dir=img_root)
  
  # Instantiate DataLoader
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=0)
  
  return test_loader

In [7]:
"""
Returns: predictions for test images
Input arguments
  net: trained network
  data_loader: DataLoader containing test data
  device: which device to use during testing phase (default is GPU)
"""
def test(net, data_loader, device='cuda:0'):
  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  predictions = {}
  with torch.no_grad():
    for batch_idx, (inputs, img_file) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)

      # Forward pass
      outputs = net(inputs)
      
      # Save predictions
      predicted = torch.round(outputs)
      predictions[img_file] = predicted

  return predictions


In [8]:
"""
Returns: refined prediction with format based on origina annotations dataframe
Input arguments
  labels: computed labels
  test_res: test result in a one-hot encoding format
"""
def refine_predictions(labels, test_res):
  predictions_refined = []
  # convert list of "label-value" to only list of "label"
  single_labels = list(set([el.split("-")[0] for el in labels.keys()]))
  for img in test_res:
    pred = list(np.concatenate(test_res[img].tolist()).flat)
    current_pred = {}
    current_pred["id"] = img[0].split(".")[0] 
    for l in single_labels:
      current_pred[l] = 1
    for key in labels:
      if (pred[labels[key]] == 1):
        current_pred[key.split("-")[0]] = key.split("-")[1]
    predictions_refined.append(current_pred)
  return predictions_refined

"""
Input arguments
  annotations_frame: origina annotations dataframe
  predictions_refined: refined prediction converted from the one-hot encoding
"""
def write_results_csv(annotations_frame, predictions_refined):
  with open('results.csv', 'w',  newline='') as f:
      # create the csv writer
      writer = csv.writer(f)
      # write the header
      writer.writerow(list(annotations_frame.keys()))
      
      for pred in predictions_refined:
        row = []
        for key in annotations_frame.keys():
          row.append(pred[key])
        writer.writerow(row)  
      

In [10]:
def main(device='cuda:0', 
         img_root='./dataset',
         model_location="/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/models/resnet50_5epoch.pth",
         batch_size=128):
  
  # Load csv with annotations into dataframe
  annotations_frame = pd.read_csv("%s/annotations_train.csv" % (img_root))

  # Get labels dict
  labels = setupLabelsDict(annotations_frame)
  num_classes = len(labels)

  # Instantiates dataloaders
  test_loader = get_data(img_root="%s/test" % (img_root))
  
  # Instantiates the model to evaluate
  net = initialize_resnet(num_classes)
  net.load_state_dict(torch.load(model_location))
  net.to(device)

  # Get test dataset predictions
  test_res = test(net, test_loader)

  # Get predictions from the one-hot encoding from the original label
  predictions_refined = refine_predictions(labels, test_res)

  # Write the prediction result to CSV
  write_results_csv(annotations_frame, predictions_refined)


In [ ]:
main()